In [4]:
import os
def filename_list(dir):
    images = []
    dir = os.path.expanduser(dir)# 扩展用户名。加上前面的部分
    # print('dir {}'.format(dir))
    for filename in os.listdir(dir):
        # print(filename)
        file_path = os.path.join(dir, filename)
        images.append(file_path)
        # print(file_path)
    # print(images)
    return images #这是返回了一个列表
file_list = filename_list('../sample/')
print(file_list)

dir ../sample/
['../sample/gt.txt', '../sample/test', '../sample/train', '../sample/val']


In [1]:
import numpy as np

# 1. 读取 .npy 文件
data = np.load('../sample/train/init1/MR_1.npy')  # 用实际文件名替代 'your_file.npy'

# 2. 查看数据的维度
print("数据的维度:", data.shape)

# 3. 查看数据的大小（元素个数）
print("数据的大小（元素个数）:", data.size)


数据的维度: (32, 96, 96)
数据的大小（元素个数）: 294912


In [2]:
import torch
import torch.nn as nn

# 假设你的输入图像大小为 (batch_size, channels, height, width)
input_size = (1, 8, 32, 32)  # 示例输入大小

# 创建一个示例的输入
x = torch.randn(input_size)

# 使用Upsample来进行上采样，以便将图像大小恢复到原始尺寸
upsample_factor = 2  # 上采样因子
upsampled_x = nn.functional.interpolate(x, scale_factor=upsample_factor, mode='nearest')

# 检查上采样后的张量大小
print("原始输入大小:", x.size())
print("上采样后的大小:", upsampled_x.size())


原始输入大小: torch.Size([1, 8, 32, 32])
上采样后的大小: torch.Size([1, 8, 64, 64])


In [3]:
import torch
import torch.nn as nn

# 假设你的输入图像大小为 (batch_size, channels, height, width)
input_size = (1, 8, 32, 32)  # 示例输入大小

# 创建一个示例的输入
x = torch.randn(input_size)

# 反卷积层用于上采样以恢复原始图像大小
# 在这个示例中，我们将上采样因子设置为2
upsample_factor = 2

# 创建一个反卷积层
deconv_layer = nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=4, stride=upsample_factor, padding=1)

# 上采样操作
upsampled_x = deconv_layer(x)

# 检查上采样后的张量大小
print("原始输入大小:", x.size())
print("上采样后的大小:", upsampled_x.size())


原始输入大小: torch.Size([1, 8, 32, 32])
上采样后的大小: torch.Size([1, 8, 64, 64])


In [2]:
import torch
import torch.nn as nn

# 假设你的输入图像大小为 (batch_size, channels, depth, height, width)
input_size = (1, 64, 16, 16, 16)  # 示例输入大小

# 创建一个示例的输入
x = torch.randn(input_size)

# # 上采样因子（在深度、高度和宽度方向上的上采样倍数）
# depth_upsample = 2
# height_upsample = 2
# width_upsample = 2

# deconv_layer = nn.ConvTranspose3d(in_channels=64, out_channels=32, kernel_size=4, stride=4, padding=0)

# upsampled_x = deconv_layer(x)

# # 看一下池化
# maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
# x = maxpool(upsampled_x)
# 检查上采样后的张量大小

conv = nn.Conv3d(in_channels=64,
            out_channels=32,
            kernel_size=1,
            stride=(1, 1, 1),
            padding=1,
            bias=False)
conv1 = conv(x)
print("原始输入大小:", x.size())
print('卷积后大小：',conv1.size())
# print("上采样后的大小:", upsampled_x.size())
# print('池化后大小：',x.size())


原始输入大小: torch.Size([1, 64, 16, 16, 16])
卷积后大小： torch.Size([1, 32, 18, 18, 18])


In [4]:


x1 = torch.randn((2,2))

x2 = torch.randn((2,2))

print(x1)
print("===========")
print(x2)
print("===========")
print(x1+x2)

tensor([[-0.8547, -0.1450],
        [-0.1727,  0.0471]])
tensor([[ 0.4295, -0.3780],
        [ 1.6908, -1.1901]])
tensor([[-0.4252, -0.5230],
        [ 1.5182, -1.1431]])


In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NLBlockND_cross(nn.Module):
    # Our implementation of the attention block referenced https://github.com/tea1528/Non-Local-NN-Pytorch
    # 这一块儿就是夸模态注意力模块
    def __init__(self, in_channels, inter_channels=None, mode='embedded',
                 dimension=3, bn_layer=True):
        """Implementation of Non-Local Block with 4 different pairwise functions but doesn't include subsampling trick
        args:
            in_channels: original channel size (1024 in the paper)
            inter_channels: channel size inside the block if not specifed reduced to half (512 in the paper)
            mode: supports Gaussian, Embedded Gaussian, Dot Product, and Concatenation
            dimension: can be 1 (temporal), 2 (spatial), 3 (spatiotemporal)
            bn_layer: whether to add batch norm
        """
        super(NLBlockND_cross, self).__init__()

        assert dimension in [1, 2, 3]

        if mode not in ['gaussian', 'embedded', 'dot', 'concatenate']:
            raise ValueError('`mode` must be one of `gaussian`, `embedded`, `dot` or `concatenate`')

        self.mode = mode
        self.dimension = dimension

        self.in_channels = in_channels
        self.inter_channels = inter_channels

        # the channel size is reduced to half inside the block
        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
            if self.inter_channels == 0:
                self.inter_channels = 1

        # assign appropriate convolutional, max pool, and batch norm layers for different dimensions
        if dimension == 3:
            conv_nd = nn.Conv3d
            max_pool_layer = nn.MaxPool3d(kernel_size=(1, 2, 2))
            bn = nn.BatchNorm3d
        elif dimension == 2:
            conv_nd = nn.Conv2d
            max_pool_layer = nn.MaxPool2d(kernel_size=(2, 2))
            bn = nn.BatchNorm2d
        else:
            conv_nd = nn.Conv1d
            max_pool_layer = nn.MaxPool1d(kernel_size=(2))
            bn = nn.BatchNorm1d

        # function g in the paper which goes through conv. with kernel size 1
        self.g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels, kernel_size=1)

        # add BatchNorm layer after the last conv layer
        # 为非局部块的最后一个卷积层添加批量归一化，以提高模型的训练稳定性，并确保初始状态是一个恒等映射
        if bn_layer:
            self.W_z = nn.Sequential(
                conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels, kernel_size=1),
                bn(self.in_channels)
            )
            # from section 4.1 of the paper, initializing params of BN ensures that the initial state of non-local block is identity mapping
            nn.init.constant_(self.W_z[1].weight, 0)
            nn.init.constant_(self.W_z[1].bias, 0)
        else:
            self.W_z = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels, kernel_size=1)

            # from section 3.3 of the paper by initializing Wz to 0, this block can be inserted to any existing architecture
            nn.init.constant_(self.W_z.weight, 0)
            nn.init.constant_(self.W_z.bias, 0)

        # define theta and phi for all operations except gaussian
        if self.mode == "embedded" or self.mode == "dot" or self.mode == "concatenate":
            self.theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels, kernel_size=1)
            self.phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels, kernel_size=1)

        if self.mode == "concatenate":
            self.W_f = nn.Sequential(
                nn.Conv2d(in_channels=self.inter_channels * 2, out_channels=1, kernel_size=1),
                nn.ReLU()
            )

    def forward(self, x_thisBranch, x_otherBranch):
        #x_thisBranch for g and theta
        #x_otherBranch for phi
        """
        args
            x: (N, C, T, H, W) for dimension=3; (N, C, H, W) for dimension 2; (N, C, T) for dimension 1
        """
        # print(x_thisBranch.shape)

        batch_size = x_thisBranch.size(0)

        # (N, C, THW)
        # this reshaping and permutation is from the spacetime_nonlocal function in the original Caffe2 implementation
        g_x = self.g(x_thisBranch).view(batch_size, self.inter_channels, -1)
        g_x = g_x.permute(0, 2, 1)

        if self.mode == "gaussian":
            theta_x = x_thisBranch.view(batch_size, self.in_channels, -1)
            phi_x = x_otherBranch.view(batch_size, self.in_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            f = torch.matmul(theta_x, phi_x)

        elif self.mode == "embedded" or self.mode == "dot":
            theta_x = self.theta(x_thisBranch).view(batch_size, self.inter_channels, -1)
            phi_x = self.phi(x_otherBranch).view(batch_size, self.inter_channels, -1)
            # theta_x = theta_x.permute(0, 2, 1)
            phi_x = phi_x.permute(0, 2, 1)
            f = torch.matmul(phi_x, theta_x)

        # elif self.mode == "concatenate":
        else: #default as concatenate
            theta_x = self.theta(x_thisBranch).view(batch_size, self.inter_channels, -1, 1)
            phi_x = self.phi(x_otherBranch).view(batch_size, self.inter_channels, 1, -1)

            h = theta_x.size(2)
            w = phi_x.size(3)
            theta_x = theta_x.repeat(1, 1, 1, w)
            phi_x = phi_x.repeat(1, 1, h, 1)

            concat = torch.cat([theta_x, phi_x], dim=1)
            f = self.W_f(concat)
            f = f.view(f.size(0), f.size(2), f.size(3))

        if self.mode == "gaussian" or self.mode == "embedded":
            f_div_C = F.softmax(f, dim=-1)
        elif self.mode == "dot" or self.mode == "concatenate":
            N = f.size(-1)  # number of position in x
            f_div_C = f / N

        y = torch.matmul(f_div_C, g_x)

        # contiguous here just allocates contiguous chunk of memory
        y = y.permute(0, 2, 1).contiguous()
        y = y.view(batch_size, self.inter_channels, *x_thisBranch.size()[2:])

        W_y = self.W_z(y)
        # residual connection
        z = W_y + x_thisBranch 

        return z

Cross = NLBlockND_cross(32)
input_size = (1, 32, 8, 32, 32)  # 示例输入大小

# 创建一个示例的输入
x1 = torch.randn(input_size)
x2 = torch.randn(input_size)

result1 = Cross(x1,x2)# 这里其实并没有改变输入的通道数
result2 = Cross(x2,x1)# 这两个的输出是一样的

print(result1.size())
print(result2.size())
# result = torch.cat((result1, result2), 1)
# print(result.size())
# print(result1)
# print('============')
# print(result2)



torch.Size([1, 32, 8, 32, 32])
torch.Size([1, 32, 8, 32, 32])


In [21]:
input_size = (1, 16, 16, 32, 32)  # 示例输入大小
x1 = torch.randn(input_size)

# pooltest = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2), padding=0)
# torch.Size([1, 16, 16, 16, 16])
# pooltest = nn.MaxPool3d(kernel_size=(1,2,2), stride=2, padding=0)
# torch.Size([1, 16, 8, 16, 16])
# pooltest = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
# torch.Size([1, 16, 8, 16, 16])
result = pooltest(x1)
print(result.size())

torch.Size([1, 16, 8, 16, 16])


In [46]:
input_size = (1, 16, 32, 128, 128)
x1 = torch.randn(input_size)
x2 = torch.randn(input_size)
test = nn.Sequential(
            # deconvolution:
            # voxelmorph里面用的是上采样最近邻插值，我们用哪一个？
            nn.ConvTranspose3d(in_channels=16, 
                               out_channels=32, 
                               kernel_size=4, # kernel_size=2的话padding=0
                               stride=2, # 在第二个里面是不是要给它放大四倍？因为前面是三个池化 不可以，因为要拼接
                               padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True)
        )
x1 = test(x1)
print(x1.size())

torch.Size([1, 32, 64, 256, 256])


In [ ]:
import os
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from deformable import *
from SPT import *
from Edge_ssim_loss import *

fixed = np.load('/media/user_gou/Elements/Shi/Attention-Reg-main/cropeddata/01_01.npy')
moving = np.load('/media/user_gou/Elements/Shi/Attention-Reg-main/cropeddata/02_01.npy')

fixed = torch.tensor(fixed,dtype=torch.float64)
moving = torch.tensor(moving,dtype=torch.float64)

print(fixed.size())
